In [ ]:


from sklearn.inspection import permutation_importance
from sklearn.calibration import CalibrationDisplay
from sklearn.preprocessing import MinMaxScaler

# --- Step 22: Feature Importances for All Models ---
print("\n--- Step 22: Feature Importances for All Models ---")

feature_importance_results = {}

for name, model in models.items():
    try:
        if hasattr(model, "feature_importances_"):
            importance_vals = model.feature_importances_
        elif hasattr(model, "coef_"):
            importance_vals = np.abs(model.coef_[0])  # Logistic Regression
        else:
            # permutation importance
            perm_result = permutation_importance(
                model, X_test_processed, y_test,
                n_repeats=10, random_state=42, n_jobs=-1
            )
            importance_vals = perm_result.importances_mean

        feature_importance_results[name] = pd.DataFrame({
            "Feature": processed_feature_names,
            f"Importance_{name}": importance_vals
        }).sort_values(by=f"Importance_{name}", ascending=False)

        print(f"\nTop 10 Features for {name}:")
        print(feature_importance_results[name].head(10))

    except Exception as e:
        print(f"Skipping feature importance for {name}: {e}")

# --- Step 23: Combined Feature Importance Plot (Grouped Bars) ---
print("\n--- Step 23: Combined Feature Importance Plot ---")

# all importance tables
merged_importances = feature_importance_results[list(feature_importance_results.keys())[0]][["Feature"]]

for name, df_imp in feature_importance_results.items():
    merged_importances = merged_importances.merge(df_imp, on="Feature", how="left")

# --- NEW: Scale all importance values per model to [0,1] ---
scaler = MinMaxScaler()
for col in merged_importances.columns:
    if col != "Feature":
        merged_importances[col] = scaler.fit_transform(merged_importances[[col]])

# Melt for plotting
merged_melted = pd.melt(
    merged_importances,
    id_vars="Feature",
    var_name="Model",
    value_name="Importance"
)

plt.figure(figsize=(18, 8))
sns.barplot(data=merged_melted, x="Feature", y="Importance", hue="Model")
plt.title("Top Normalized Features across Models")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

from sklearn.calibration import CalibrationDisplay
import matplotlib.pyplot as plt

print("\n--- Step 24: Calibration Curves ---")

# Define 12 distinct colors
colors = [
    "#1f77b4" ,"#7f7788", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b",
    "#e377c2", "#7f7f7f", "#bcbd22", "#17becf", "#aec7e8", "#fcbb70"
]

plt.figure(figsize=(10, 8))

for i, (name, y_prob) in enumerate(prob_maps.items()):
    color = colors[i % len(colors)]  # cycle colors if more than 12
    try:
        CalibrationDisplay.from_predictions(
            y_test, y_prob, n_bins=10, strategy='uniform',
            name=name, ax=plt.gca(), color=color
        )
    except Exception as e:
        print(f"Skipping calibration for {name}: {e}")

plt.plot([0, 1], [0, 1], "k--", label="Perfectly Calibrated")
plt.title("Calibration Curves for All Models")
plt.xlabel("Mean Predicted Probability")
plt.ylabel("Fraction of Positives")
plt.legend(loc="best")
plt.grid(True)
plt.show()


